In [4]:
using DataFrames, CSV
using JuMP, Gurobi
using LinearAlgebra
using JSON

In [7]:
config = JSON.parsefile("config.json")

distance = Matrix(CSV.read("inputs/distance_matrix.csv", DataFrame, header = true))
demand_raw = Matrix(CSV.read("inputs/salem_demand.csv", DataFrame, header = true))
demand = demand_raw[:, 9];


In [6]:
# parameters

N_CANDIDATE_STATIONS = size(distance)[1]
N_NEW_STATIONS = config["N_NEW_STATIONS"]
DISTANCE_THRESHOLD = config["DISTANCE_THRESHOLD"]
N_CURRENT_STATIONS = size(demand)[1]

LoadError: UndefVarError: demand not defined

In [40]:
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 0) 

M = 1_000_000

# @variable(model, x[i=1:50, j=1:15] >= 0) # amount of waste transported
@variable(model, z[i=1:N_CANDIDATE_STATIONS], Bin) # z: binary variable to indicate whether or not station is selected

# @constraint(model, [i=1:50], sum(x[i, j] for j in 1:15) == q[i])
# @constraint(model, [i=1:50, j=1:15], x[i, j] <= M * z[j])

# add constraint that two candidate stations cannot be too close to each other or existing stations

@constraint(model, sum(z[i] for i in 1:N_CANDIDATE_STATIONS) == N_NEW_STATIONS)
@constraint(model, [i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS], distance[i, j] >=  1000 * z[i])

@objective(model, Max, sum(-distance[i, j] * demand[j] * z[i] for i=1:N_CANDIDATE_STATIONS, j=1:N_CURRENT_STATIONS))

optimize!(model)

optimal_cost = objective_value(model)
stations_chosen = value.(z)


println("Optimal stations: $stations_chosen")


# printing optimal cost
pretty_optimal_cost = round(optimal_cost)
println("Total Cost: $pretty_optimal_cost")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-16
Optimal stations: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0]
Total Cost: -3.013348e6


In [41]:
candidate_station_addresses[findall(x -> x==1, stations_chosen)]

5-element Vector{Any}:
 "Unnamed Road, Salem, MA 01970, USA"
 "91 Leach St, Salem, MA 01970, USA"
 "Building 4, Suite 160 Shetland Park, 45 Congress St, Salem, MA 01970, USA"
 "Unnamed Road, Salem, MA 01970, USA"
 "98 Collins St, Salem, MA 01970, USA"

In [29]:
findall(stations_chosen)

LoadError: TypeError: non-boolean (Float64) used in boolean context